# Text generation with an RNN
This notebook is heavily based on the excellent blog ["The Unreasonable Effectiveness of Recurrent Neural Networks"](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) by Andrej Karpathy (which you must read) and the Keras example ["Character-level text generation with LSTM"](https://keras.io/examples/generative/lstm_character_level_text_generation/) by François Chollet.

In [1]:
import keras
from keras import layers
import numpy as np
import random
import io

## Load a text from [Project Gutenberg](https://www.gutenberg.org/)
Project Gutenberg is a library of over 70,000 free eBooks

Here we shall load "*Alice's Adventures in Wonderland*" by Lewis Carroll

In [2]:
url = 'https://www.gutenberg.org/cache/epub/11/pg11.txt'
path = keras.utils.get_file(origin=url)

with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # We remove newline chars for nicer display
print("Corpus length:", len(text))

174355/174355 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step
Corpus length: 163916


In [3]:
chars = sorted(list(set(text)))
print("Number unique characters:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 64
step = 3

sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype="bool")
y = np.zeros((len(sentences), len(chars)), dtype="bool")

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number unique characters: 64
Number of sequences: 54618


# A very simple LSTM model

In [4]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)

In [5]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)

model.compile(loss="categorical_crossentropy",
              optimizer=optimizer)

# Fit (train) the model

In [6]:
%%time

epochs = 10
batch_size = 128

model.fit(x, y,
          batch_size=batch_size,
          epochs=epochs)

Epoch 1/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 69s 156ms/step - loss: 2.6886
Epoch 2/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 66s 155ms/step - loss: 1.8968
Epoch 3/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 65s 152ms/step - loss: 1.6436
Epoch 4/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 64s 150ms/step - loss: 1.5011
Epoch 5/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 65s 151ms/step - loss: 1.4087
Epoch 6/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 65s 151ms/step - loss: 1.3074
Epoch 7/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 65s 152ms/step - loss: 1.2498
Epoch 8/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 65s 153ms/step - loss: 1.1906
Epoch 9/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 64s 151ms/step - loss: 1.1481
Epoch 10/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 65s 152ms/step - loss: 1.1042
CPU times: user 16min 50s, sys: 38.5 s, total: 17min 29s
Wall time: 11min 10s


# Predict for the temperatures `[0.01, 0.5, 0.7, 1]`

In [7]:
def sample(softmax, temperature=1.0): # default temperature is 1
    softmax = np.asarray(softmax).astype("float64")
    logits = np.log(softmax) / temperature # apply temperature scaling
    exp_preds = np.exp(logits)
    softmax = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, softmax, 1)
    return np.argmax(probas)

number_of_letters_to_generate = 512
# pick a random starting point from somewhere in the text
start_index = random.randint(0, len(text) - maxlen - 1)

for temperature in [0.01, 0.5, 0.7, 1]:  #  the hotter the more nonsense
    print("Temperature:", temperature)

    #sentence = text[start_index : start_index + maxlen]
    # this particular seed sentence is specific to "Alice's Adventures in Wonderland"
    sentence = "after time she heard a little pattering of feet in the distance " # len = 64, i.e. maxlen

    generated = ""
    for i in range(number_of_letters_to_generate):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.0
        preds = model.predict(x_pred, verbose=0)[0] # returns softmax
        next_index = sample(preds, temperature)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char

    print(generated)
    print("")

Temperature: 0.01
of the sabbit all the same was been to say the court was a great of the sabbit all the same was been to say the court was a great of the sabbit all the same was been to say the court was a great of the sabbit all the same was been to say the court was a great of the sabbit all the same was been to say the court was a great of the sabbit all the same was been to say the court was a great of the sabbit all the same was been to say the court was a great of the sabbit all the same was been to say the court was 

Temperature: 0.5
of liciuse, to the two stain, and began sortan as the was engloding in at of and reseration of course,” the modaration ordered of the used out of them, and sometige to make on of sausent, and then the court!” and she to knower and how down to say the other of setter and come, and the trademanimakion, how promise of the sadden up a donations and wast the end, and the poor with a minutes and this was to the used and just be a convertations of the sa